In [5]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp2...


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43899)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [3]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


성장형 무기 소지자가, 해당 스킨을 안쓰고 다른 스킨을 쓰는 경우가 얼마나 되는지입니다. (그리고 트렌드도...)
성장형 스킨을 뽑은 사람은 잠시 그것을 쓰겠지만, 철이 지나고 나면 다른 일반 고급스킨으로 갈아타는 패턴이 생겼으면 좋겠다고 생각하는데, 과연 그렇게 흐르는지 알고 싶어요.
성장형 무기가 장기적으로 다른 스킨 매출에 영향을 주는지를 유추해볼 수 있지 않을까 싶습니다.

In [6]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [10]:
gcoin = pre_load_data_mart(start_date="2021-06-02", end_date="2021-06-29", device="pc", table_name="gcoin_use")

In [11]:
# 6월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-06-02'
end_date = '2021-06-15' 
item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [2]:
# 6월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-06-02'
end_date = '2021-06-29' 
item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [13]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales")
wsus_product_id = items.where(lower(col("product_name")).like("%progressive%")).select(col("product_id")).distinct().toPandas()["product_id"].tolist()
weapon_product_list = items.where(col("sub_category") == "weapon").select(col("product_id")).distinct().toPandas().product_id.tolist()

In [14]:
wsus_product_id

['itemdesc.12012001',
 'itemdesc.12012004',
 'itemdesc.12012002',
 'itemdesc.12012003']

In [15]:
# 성장형 무기를 획득한 사람들, 
# ItemDescId, AccountId
wsus_account = item_upgrade.where(col("ItemDescId").isin(wsus_product_id)).select("ItemDescId", "AccountId").distinct().toPandas()

In [16]:
wsus_account.shape

(116538, 2)

In [17]:
wsus_account.head()

,ItemDescId,AccountId
0,itemdesc.12012001,account.4e30bc6823dd45e5b5a38337ef49ce45
1,itemdesc.12012001,account.027a8d23f335458b9574234580e5d02c
2,itemdesc.12012001,account.6a1383eb88ee44a68c8313ef6b4574fa
3,itemdesc.12012001,account.b4e7a2edb814421ca46cc57e6113e877
4,itemdesc.12012001,account.748e6d493e014340acfd9121e99013a4


In [18]:
june_wsus_account_list = wsus_account[wsus_account.ItemDescId == "itemdesc.12012001"].AccountId.tolist()

In [19]:
len(june_wsus_account_list)

116538

In [20]:
wsus_account[wsus_account.AccountId.isin(june_wsus_account_list)].groupby("AccountId", as_index=False)["ItemDescId"].count().describe()

,ItemDescId
count,116538.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [21]:
# 성장형 무기 보유자들의 성장형 무기 보유 개수
wsus_cnt = item_upgrade.where((col("ItemDescId").isin(wsus_product_id))).groupBy("AccountId").agg(countDistinct("ItemDescId")).toPandas()

In [22]:
wsus_cnt.describe()

,count(DISTINCT ItemDescId)
count,116538.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [23]:
weapon_product_list = items.where(col("sub_category") == "weapon").select(col("product_id")).distinct().toPandas().product_id.tolist()

In [26]:
tmp = item_equipped.where((col("AccountId").isin(june_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [27]:
tmp.head()

,date,AccountId,ItemId,PartId,PlatformProvider,Provider,Time,Type
0,2021-06-02,account.c50c67b9e39a43a8ba253e1b024468ef,itemdesc.12010395,partdesc.mosin_nagant,PcSteam,steam,2021-06-02T11:03:44.3835211Z,ItemEquipped
1,2021-06-02,account.bba3089d23384ae79849b2ffb641b706,itemdesc.12010409,partdesc.m762,PcSteam,steam,2021-06-02T11:03:48.6706498Z,ItemEquipped
2,2021-06-02,account.6219fae014e942a88178c60512b2dfa8,itemdesc.12012001,partdesc.m762,PcSteam,steam,2021-06-02T11:03:49.3623573Z,ItemEquipped
3,2021-06-02,account.17b9a9414f814ffe89ce6e52c20ac52d,itemdesc.12012001,partdesc.m762,PcSteam,steam,2021-06-02T11:03:53.1683234Z,ItemEquipped
4,2021-06-02,account.0d5fb8fca97744cd9d9c74c65638e803,itemdesc.12012001,partdesc.m762,PcSteam,steam,2021-06-02T11:03:54.1519489Z,ItemEquipped


In [28]:
june_wsus_equip_account_list = tmp[(tmp.AccountId.isin(june_wsus_account_list)) & (tmp.ItemId == "itemdesc.12012001")].AccountId.unique().tolist()

In [29]:
len(june_wsus_equip_account_list)

115653

115653명 대하여 기간동안 사용한 무기 스킨 수

In [30]:
tmp[tmp.AccountId.isin(june_wsus_equip_account_list)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,115653.000000
mean,2.708352
std,2.826284
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,68.000000


In [31]:
# 6월 성장형 무기 소유자 & 이용자(115653명)에 대한 무기에 대해 소비
gcoin_use = gcoin.where((col("account_id").isin(june_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).groupBy("account_id", "product_id").agg((sum("paid_use")/100).alias("paid")).toPandas()

In [32]:
gcoin_use.to_csv("./data/june_gcoin_use.csv", index=False)

그래프 용
- 일별 사용 무기 스킨
- 무기 스킨별 매출
- slr 무기 스킨별 매출

In [33]:
june_wsus_item_equipped = item_equipped.where((col("AccountId").isin(june_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).groupBy("date", "AccountId", "ItemId").agg(count("*").alias("cnt"))
product_name = items.select("product_id", "product_name").distinct()
june_wsus_item_equipped_table = june_wsus_item_equipped.join(product_name, june_wsus_item_equipped.ItemId == product_name.product_id, "left")
mysql.insert_table(june_wsus_item_equipped_table, "labs", "june_wsus_item_equipped_table")

In [34]:
june_wsus_item_equipped_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- cnt: long (nullable = false)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)



In [35]:
june_wsus_item_equipped_table.groupBy("product_name").agg(sum("cnt").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Gear Head - Beryl M762,281086
1,Gold Plate - Beryl M762,41822
2,CNH's SLR,20615
3,Silver Plate - M416,11161
4,Gold Plate - M416,10845
5,Silver Plate - Beryl M762,10842
6,PGI.S Hydrodip - M416,9083
7,MOZZ's M24,8786
8,MinORu's Mini14,7683
9,Seahorse Cult - Beryl M762,6664


In [36]:
june_wsus_item_equipped_table.groupBy("product_name").agg(countDistinct("AccountId").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Gear Head - Beryl M762,115653
1,CNH's SLR,19644
2,Gold Plate - Beryl M762,18327
3,MOZZ's M24,7120
4,Gold Plate - M416,6407
5,MinORu's Mini14,6194
6,PGI.S Hydrodip - M416,5630
7,PCS4 Gold Mettle - Pan,5332
8,Silver Plate - M416,5056
9,Silver Plate - Beryl M762,4574


In [38]:
june_wsus_gcoin_table = gcoin.where((col("account_id").isin(june_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(june_wsus_gcoin_table, "labs", "june_wsus_gcoin_table")

In [39]:
june_wsus_gcoin_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- free_use: long (nullable = true)
 |-- paid_use: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_name: string (nullable = true)



In [40]:
june_wsus_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,CNH's SLR,84536.4
1,MinORu's Mini14,20837.9
2,MOZZ's M24,15291.1
3,Harlequin - SLR,14240.4
4,Manticore - SLR,12070.9
5,Colorful Chaos - Mk14,4882.7
6,Seahorse Cult - Beryl M762,3668.1
7,Manticore - AKM,3350.8
8,Harlequin - AKM,2591.0
9,Blue Whale Cult - AKM,1794.5


In [49]:
june_m762_gcoin_table = gcoin.where((col("account_id").isin(june_wsus_account_list)) & (col("product_id").isin(m762_skin_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(june_m762_gcoin_table, "labs", "june_m762_gcoin_table")

In [50]:
june_m762_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,Seahorse Cult - Beryl M762,3687.9
1,Gold Plate - Beryl M762,24.7


In [51]:
june_m762_gcoin_table.groupBy("product_name").agg(countDistinct("account_id").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,Seahorse Cult - Beryl M762,402
1,Gold Plate - Beryl M762,3


In [43]:
tmp2 = pd.merge(tmp, gcoin_use, left_on=["AccountId", "ItemId"], right_on=["account_id", "product_id"], how="left").drop(["PlatformProvider", "Provider", "Type", "account_id", "product_id"], axis=1)

In [44]:
tmp2.to_csv('./data/june_tmp2.csv', index=False)

6월 성장형 무기 사용한 사람들 대상 (aug_wsus_equip_account_list) m762무기에 대한 스킨 착용

In [46]:
m762 = tmp2[(tmp2.AccountId.isin(june_wsus_equip_account_list)) & (tmp2.PartId == "partdesc.m762")].sort_values(["AccountId", "date"]).groupby(["date", "AccountId", "ItemId"], as_index=False)["paid"].sum().sort_values(["AccountId", "date"])

In [47]:
m762_skin_list = m762.ItemId.unique().tolist()

In [52]:
items.where(col("product_id").isin(m762_skin_list)).select("product_name", "product_id").distinct().show(truncate=False)

+------------------------------------+-----------------+
|product_name                        |product_id       |
+------------------------------------+-----------------+
|Happy Pew Year - Beryl M762         |itemdesc.12010355|
|Edberg's Beryl M762                 |itemdesc.12010172|
|Gold Plate - Beryl M762             |itemdesc.12010408|
|Silver Plate - Beryl M762           |itemdesc.12010409|
|Seahorse Cult - Beryl M762          |itemdesc.12010315|
|chocoTaco's Beryl M762              |itemdesc.12010132|
|[PROGRESSIVE] Gear Head - Beryl M762|itemdesc.12012001|
|17SHOU's Beryl M762                 |itemdesc.12010360|
+------------------------------------+-----------------+



In [53]:
m762.groupby(["AccountId"])["ItemId"].nunique().reset_index().describe()

,ItemId
count,115653.000000
mean,1.290853
std,0.603417
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,7.000000


In [54]:
m762.groupby(["AccountId"])["ItemId"].nunique().reset_index().groupby("ItemId", as_index=False)["AccountId"].count()

,ItemId,AccountId
0,1,89066
1,2,21191
2,3,4103
3,4,989
4,5,248
5,6,54
6,7,2


m762 스킨 1개 사용한 사람은 12012001만 사용한게 맞는지 확인

In [55]:
a = m762.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [56]:
m762[m762.AccountId.isin(a[a.ItemId == 1].AccountId.unique().tolist())].ItemId.unique()

array(['itemdesc.12012001'], dtype=object)

In [57]:
mskin_cnt = m762.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [59]:
mskin2 = mskin_cnt[mskin_cnt.ItemId == 2].AccountId.unique().tolist()
mskin3 = mskin_cnt[mskin_cnt.ItemId == 3].AccountId.unique().tolist()
mskin4 = mskin_cnt[mskin_cnt.ItemId == 4].AccountId.unique().tolist()
mskin5 = mskin_cnt[mskin_cnt.ItemId == 5].AccountId.unique().tolist()
mskin6 = mskin_cnt[mskin_cnt.ItemId >= 6].AccountId.unique().tolist()

In [60]:
m762[(m762.AccountId.isin(mskin2)) & (m762.paid>0)].AccountId.nunique()

337

In [61]:
m762[(m762.AccountId.isin(mskin2)) & (m762.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,337.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [62]:
m762[(m762.AccountId.isin(mskin2)) & (m762.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010315    334
itemdesc.12010408      3
Name: AccountId, dtype: int64

In [63]:
m762[(m762.AccountId.isin(mskin3)) & (m762.paid>0)].AccountId.nunique()

32

In [64]:
m762[(m762.AccountId.isin(mskin3)) & (m762.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,32.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [65]:
m762[(m762.AccountId.isin(mskin3)) & (m762.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010315    32
Name: AccountId, dtype: int64

In [66]:
m762[(m762.AccountId.isin(mskin4)) & (m762.paid>0)].AccountId.nunique()

6

In [67]:
m762[(m762.AccountId.isin(mskin4)) & (m762.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,6.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [68]:
m762[(m762.AccountId.isin(mskin4)) & (m762.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010315    6
Name: AccountId, dtype: int64

In [69]:
m762[(m762.AccountId.isin(mskin5)) & (m762.paid>0)].AccountId.nunique()

1

In [70]:
m762[(m762.AccountId.isin(mskin5)) & (m762.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,1.0
mean,1.0
std,NaN
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [71]:
m762[(m762.AccountId.isin(mskin5)) & (m762.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010315    1
Name: AccountId, dtype: int64

In [72]:
m762[(m762.AccountId.isin(mskin6)) & (m762.paid>0)].AccountId.nunique()

1

In [73]:
m762[(m762.AccountId.isin(mskin6)) & (m762.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,1.0
mean,1.0
std,NaN
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [74]:
m762[(m762.AccountId.isin(mskin6)) & (m762.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010315    1
Name: AccountId, dtype: int64

In [4]:
334+32+8

374